In [1]:
from zaber_device import ZaberDevice
from time import sleep
import ipywidgets as widgets
from IPython.display import display,clear_output
import sys
sys.version

'2.7.13 | packaged by conda-forge | (default, May  2 2017, 13:29:36) \n[GCC 4.2.1 Compatible Apple LLVM 6.1.0 (clang-602.0.53)]'

In [3]:
def move_mm(millimeters):
    steps = int(millimeters*1e-3/(nm_resolution*1e-9))
    print "(",steps,"steps)"
    dev.move_relative(steps)

def openDevice(b):
    clear_output()
    global dev
    dev = ZaberDevice(port='/dev/tty.usbserial-A403L6SM') # Mac OS X
    # dev = ZaberDevice(port='COM94') # Windows
    print("Connected to {}".format(dev.get_actuator_id()[0]))
    global onDur
    global offDur
    onDur = 5.1
    offDur = 2
    dev._send_request(37,0,128) # set resolution to 128
    print ('set to {} microsteps'.format(dev._get_microstep_resolution()[0]))
    global nm_resolution
    nm_resolution = 0.124023437*64/dev._get_microstep_resolution()[0]*1000
    actualSpeed = nm_resolution*9.375*3600/(10**6) # mm/hr
    speedWithDelay = actualSpeed*onDur/(onDur+offDur)
    print("{} nm resolution\nminimum speed: {} mm/hr\nspeed with delay: {} mm/hr\n").format(nm_resolution,actualSpeed,speedWithDelay)


w = widgets.BoundedFloatText(
    value=2,
    min=0,
    max=9000,
    step=0.1,
    disabled=False,
    layout=widgets.Layout(width='75px')
)
lbl = widgets.Label(value="mm")

up=widgets.Button(
    description='UP',
)
down=widgets.Button(
    description='DOWN',
)
homeBtn=widgets.Button(
    description='HOME',
)
stopBtn=widgets.Button(
    description='STOP',
)

openBtn=widgets.Button(
    description='OPEN serial',
)

closeBtn=widgets.Button(
    description='CLOSE serial',
)

positionBtn=widgets.Button(
    description='Get Position',
)

def sendUp(b):
    print -w.value,"mm",
    move_mm(-w.value)
def sendDown(b):
    print(w.value),"mm",
    move_mm(w.value)
def goHome(b):
    print "Homing..."
    dev.home()    
def stopMotor(b):
    dev.stop()
def closeDevice(b):
    dev.close()
    print "device closed"
def getPos(b):
    print(dev.get_position()[0])
    
up.on_click(sendUp)
down.on_click(sendDown)
homeBtn.on_click(goHome)
stopBtn.on_click(stopMotor)
openBtn.on_click(openDevice)
closeBtn.on_click(closeDevice)
positionBtn.on_click(getPos)

connection = widgets.VBox([openBtn,closeBtn])
inputBox = widgets.HBox([w,lbl])
colm1 = widgets.VBox([up,inputBox,down])
colm2 = widgets.VBox([homeBtn,positionBtn,stopBtn])
display(widgets.HBox([connection,colm1,colm2]))

openDevice(1)

Connected to 6721
set to 128 microsteps
62.0117185 nm resolution
minimum speed: 2.09289549937 mm/hr
speed with delay: 1.50334747138 mm/hr



In [23]:
while(1):    
    dev.move_at_speed(-1)
    sleep(onDur)
    dev.stop()
    sleep(offDur)
# type I,I to interrupt kernel, then press stop button to stop the movement